In [76]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
        return resp

In [63]:
url = 'http://openapi.its.go.kr:8081/api/NCCTVInfo'
api_key = '1602398376506'
params= {
    'key':api_key,
    'ReqType':'2',
    'type ':'ex',
    'MinX' : 127.100000,
    'MaxX' : 128.890000,
    'MinY' : 34.100000,
    'MaxY' : 39.100000
}
resp=download(url,params, headers,'POST')

In [64]:
dom = BeautifulSoup(resp.text, 'html.parser')

In [65]:
col = ['roadsectionid','filecreatetime','cctvtype','cctvurl','cctvresolution','coordy','cctvformat','cctvname','coordx']
row = []
rows = []
for i in dom.find_all('data'):
    for co in col:
        k = i.find(co).text.strip()
        row.append(k)
    rows.append(row)
    row = []

In [66]:
import pandas as pd
data = pd.DataFrame(rows,columns=col)
data.head()

,roadsectionid,filecreatetime,cctvtype,cctvurl,cctvresolution,coordy,cctvformat,cctvname,coordx
0,,,2,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,,37.42889,MP4,[서울외곽선] 성남,127.12361
1,,,2,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,,37.43909,MP4,[서울외곽선] 성남영업소,127.12262
2,,,2,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,,37.475,MP4,[서울외곽선] 송파,127.12944
3,,,2,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,,37.51167,MP4,[서울외곽선] 서하남,127.14972
4,,,2,http://cctvsec.ktict.co.kr/6/ImX13W+ZpBe0tusyH...,,37.51527778,MP4,[서울외곽선] 광암터널2,127.1652778


In [67]:
for i in col:
    print(sum(data[i]==''))

2331
2331
0
0
2331
0
0
0
0


In [68]:
data = data.drop(['roadsectionid','filecreatetime','cctvresolution','cctvformat','cctvtype'],axis=1)
data.head()

,cctvurl,coordy,cctvname,coordx
0,http://cctvsec.ktict.co.kr/2/Do71Nvv+HamcAfOiT...,37.42889,[서울외곽선] 성남,127.12361
1,http://cctvsec.ktict.co.kr/3/9sQQra/DP1w1VIjZu...,37.43909,[서울외곽선] 성남영업소,127.12262
2,http://cctvsec.ktict.co.kr/4/Ishh8oNCksfXOlFzQ...,37.475,[서울외곽선] 송파,127.12944
3,http://cctvsec.ktict.co.kr/5/azcq8e7d5lOPfYjjx...,37.51167,[서울외곽선] 서하남,127.14972
4,http://cctvsec.ktict.co.kr/6/ImX13W+ZpBe0tusyH...,37.51527778,[서울외곽선] 광암터널2,127.1652778


In [92]:
data['cctvurl'][470]

'http://cctvsec.ktict.co.kr/2345/TckwwQSiuA6VAOYzZ5Wed+xA6MEnFejP/yIXw9bsEumvqYck2H0rEHLO5xoPOowd'

In [69]:
data.to_csv('CCTV.csv',index=False)

In [90]:
import urllib
for url,name in zip(data['cctvurl'],data['cctvname']):
    urllib.request.urlretrieve(url,'data/'+ name + '.mp4')

FileNotFoundError: [Errno 2] No such file or directory: 'data/[서울양양선] [양양]인제양양터널(115/115).mp4'

In [80]:
CCTV

<Response [400]>

In [57]:
import seaborn as sns
data['cctvtype'].value_counts()

2    2332
Name: cctvtype, dtype: int64

In [70]:
url = 'https://www.weather.go.kr/cgi-bin/aws/nph-aws_txt_min_guide_test?202010111514&-60&MINDB_01M&0&m&K'
resp=download(url,  headers = headers,method= 'POST')

In [71]:
dom = BeautifulSoup(resp.content, 'lxml')

In [72]:
k =pd.read_html(str(dom.find('table')),header=0)
len(k)

2

In [73]:
k[1].to_csv('AWS_measure.csv',index=False)